In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.12.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid() :
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(sigmoid())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(sigmoid())

    model.add(dense(label_dim, weight_init))

    return model

In [10]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [11]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [12]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\jihyu\anaconda3\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 0.6926, train_loss: 2.25516844, train_accuracy: 0.1719, test_Accuracy: 0.0892
Epoch: [ 0] [    1/  468] time: 0.8707, train_loss: 2.28419089, train_accuracy: 0.1406, test_Accuracy: 0.0892
Epoch: [ 0] [    2/  468] time: 1.0477, train_loss: 2.28738451, train_accuracy: 0.0938, test_Accuracy: 0.0892
Epoch: [ 0] [    3/  468] time: 1.2319, train_loss: 2.25234318, train_accuracy: 0.1875, test_Accuracy: 0.1733
Epoch: [ 0] [    4/  468] time: 1.4150, train_loss: 2.29301500, train_accuracy: 0.1953, test_Accuracy: 0.1652
Epoch: [ 0] [    5/  468] time: 1.6033, train_loss: 2.25106192, train_accuracy: 0.3047, test_Accuracy: 0.3077
Epoch: [ 0] [    6/  468] time: 1.7844, train_loss: 2.24579453, train_accuracy: 0.3203, test_Accuracy: 0.2799
Epoch: [ 0] [    7/  468] time: 1.9790, train_loss: 2.20981288, train_accuracy: 0.3516, test_Accuracy: 0.2661
Epoch: [ 0] [    8/  468] time: 2.1756, train_loss: 2.26026535, train_accuracy: 0.2031, test_Accuracy: 0.1967
Epoch: [ 0

Epoch: [ 0] [   75/  468] time: 13.7850, train_loss: 0.92673641, train_accuracy: 0.7266, test_Accuracy: 0.7669
Epoch: [ 0] [   76/  468] time: 13.9659, train_loss: 0.91716373, train_accuracy: 0.8047, test_Accuracy: 0.7756
Epoch: [ 0] [   77/  468] time: 14.1599, train_loss: 0.91149789, train_accuracy: 0.8594, test_Accuracy: 0.7842
Epoch: [ 0] [   78/  468] time: 14.3290, train_loss: 0.86295360, train_accuracy: 0.8359, test_Accuracy: 0.7915
Epoch: [ 0] [   79/  468] time: 14.5050, train_loss: 0.94289184, train_accuracy: 0.7344, test_Accuracy: 0.7989
Epoch: [ 0] [   80/  468] time: 14.6830, train_loss: 0.79286528, train_accuracy: 0.8281, test_Accuracy: 0.8019
Epoch: [ 0] [   81/  468] time: 14.8556, train_loss: 0.87998790, train_accuracy: 0.7656, test_Accuracy: 0.8044
Epoch: [ 0] [   82/  468] time: 15.0294, train_loss: 0.85716903, train_accuracy: 0.8125, test_Accuracy: 0.8089
Epoch: [ 0] [   83/  468] time: 15.2047, train_loss: 0.85606003, train_accuracy: 0.8047, test_Accuracy: 0.8131
E

Epoch: [ 0] [  149/  468] time: 26.7235, train_loss: 0.50684202, train_accuracy: 0.8828, test_Accuracy: 0.8765
Epoch: [ 0] [  150/  468] time: 26.8978, train_loss: 0.50225490, train_accuracy: 0.8594, test_Accuracy: 0.8787
Epoch: [ 0] [  151/  468] time: 27.0799, train_loss: 0.45236862, train_accuracy: 0.8906, test_Accuracy: 0.8815
Epoch: [ 0] [  152/  468] time: 27.2443, train_loss: 0.46955517, train_accuracy: 0.8984, test_Accuracy: 0.8839
Epoch: [ 0] [  153/  468] time: 27.4651, train_loss: 0.53573400, train_accuracy: 0.8516, test_Accuracy: 0.8850
Epoch: [ 0] [  154/  468] time: 27.6602, train_loss: 0.37231386, train_accuracy: 0.9141, test_Accuracy: 0.8854
Epoch: [ 0] [  155/  468] time: 27.8681, train_loss: 0.43098441, train_accuracy: 0.8594, test_Accuracy: 0.8853
Epoch: [ 0] [  156/  468] time: 28.0472, train_loss: 0.42116541, train_accuracy: 0.8906, test_Accuracy: 0.8838
Epoch: [ 0] [  157/  468] time: 28.2493, train_loss: 0.46909130, train_accuracy: 0.8984, test_Accuracy: 0.8828
E

Epoch: [ 0] [  224/  468] time: 39.7288, train_loss: 0.35010993, train_accuracy: 0.8828, test_Accuracy: 0.8970
Epoch: [ 0] [  225/  468] time: 39.9439, train_loss: 0.36715525, train_accuracy: 0.8906, test_Accuracy: 0.8968
Epoch: [ 0] [  226/  468] time: 40.1208, train_loss: 0.38015836, train_accuracy: 0.9062, test_Accuracy: 0.8976
Epoch: [ 0] [  227/  468] time: 40.2865, train_loss: 0.42820007, train_accuracy: 0.8984, test_Accuracy: 0.8991
Epoch: [ 0] [  228/  468] time: 40.4473, train_loss: 0.30791697, train_accuracy: 0.9531, test_Accuracy: 0.8988
Epoch: [ 0] [  229/  468] time: 40.6293, train_loss: 0.40808755, train_accuracy: 0.8828, test_Accuracy: 0.8986
Epoch: [ 0] [  230/  468] time: 40.7992, train_loss: 0.30443826, train_accuracy: 0.8984, test_Accuracy: 0.8987
Epoch: [ 0] [  231/  468] time: 40.9715, train_loss: 0.30037504, train_accuracy: 0.8906, test_Accuracy: 0.8979
Epoch: [ 0] [  232/  468] time: 41.1496, train_loss: 0.45470533, train_accuracy: 0.8672, test_Accuracy: 0.8971
E

Epoch: [ 0] [  299/  468] time: 53.7572, train_loss: 0.40592715, train_accuracy: 0.9141, test_Accuracy: 0.9082
Epoch: [ 0] [  300/  468] time: 53.9667, train_loss: 0.34974343, train_accuracy: 0.9141, test_Accuracy: 0.9093
Epoch: [ 0] [  301/  468] time: 54.1511, train_loss: 0.19714314, train_accuracy: 0.9375, test_Accuracy: 0.9109
Epoch: [ 0] [  302/  468] time: 54.3448, train_loss: 0.23062865, train_accuracy: 0.9453, test_Accuracy: 0.9106
Epoch: [ 0] [  303/  468] time: 54.5181, train_loss: 0.41291726, train_accuracy: 0.9141, test_Accuracy: 0.9105
Epoch: [ 0] [  304/  468] time: 54.7451, train_loss: 0.32860622, train_accuracy: 0.9062, test_Accuracy: 0.9105
Epoch: [ 0] [  305/  468] time: 54.9571, train_loss: 0.39342368, train_accuracy: 0.8750, test_Accuracy: 0.9098
Epoch: [ 0] [  306/  468] time: 55.1479, train_loss: 0.34632325, train_accuracy: 0.8984, test_Accuracy: 0.9099
Epoch: [ 0] [  307/  468] time: 55.3306, train_loss: 0.32910800, train_accuracy: 0.8906, test_Accuracy: 0.9086
E

Epoch: [ 0] [  373/  468] time: 67.3125, train_loss: 0.21268088, train_accuracy: 0.9297, test_Accuracy: 0.9177
Epoch: [ 0] [  374/  468] time: 67.4834, train_loss: 0.33842346, train_accuracy: 0.8828, test_Accuracy: 0.9182
Epoch: [ 0] [  375/  468] time: 67.6471, train_loss: 0.37090021, train_accuracy: 0.8750, test_Accuracy: 0.9175
Epoch: [ 0] [  376/  468] time: 67.7980, train_loss: 0.22138217, train_accuracy: 0.9297, test_Accuracy: 0.9180
Epoch: [ 0] [  377/  468] time: 67.9764, train_loss: 0.37536308, train_accuracy: 0.9062, test_Accuracy: 0.9176
Epoch: [ 0] [  378/  468] time: 68.1395, train_loss: 0.25986958, train_accuracy: 0.9297, test_Accuracy: 0.9178
Epoch: [ 0] [  379/  468] time: 68.3042, train_loss: 0.20345011, train_accuracy: 0.9609, test_Accuracy: 0.9178
Epoch: [ 0] [  380/  468] time: 68.4726, train_loss: 0.30359370, train_accuracy: 0.9297, test_Accuracy: 0.9183
Epoch: [ 0] [  381/  468] time: 68.6612, train_loss: 0.40061235, train_accuracy: 0.8828, test_Accuracy: 0.9182
E

Epoch: [ 0] [  447/  468] time: 80.1478, train_loss: 0.18281427, train_accuracy: 0.9609, test_Accuracy: 0.9218
Epoch: [ 0] [  448/  468] time: 80.3105, train_loss: 0.26301348, train_accuracy: 0.9375, test_Accuracy: 0.9212
Epoch: [ 0] [  449/  468] time: 80.4885, train_loss: 0.33454114, train_accuracy: 0.8906, test_Accuracy: 0.9214
Epoch: [ 0] [  450/  468] time: 80.6548, train_loss: 0.25377151, train_accuracy: 0.9062, test_Accuracy: 0.9220
Epoch: [ 0] [  451/  468] time: 80.8155, train_loss: 0.21610101, train_accuracy: 0.9297, test_Accuracy: 0.9218
Epoch: [ 0] [  452/  468] time: 80.9868, train_loss: 0.32850808, train_accuracy: 0.9297, test_Accuracy: 0.9215
Epoch: [ 0] [  453/  468] time: 81.1690, train_loss: 0.23021629, train_accuracy: 0.9297, test_Accuracy: 0.9218
Epoch: [ 0] [  454/  468] time: 81.3577, train_loss: 0.36856300, train_accuracy: 0.9141, test_Accuracy: 0.9225
Epoch: [ 0] [  455/  468] time: 81.5339, train_loss: 0.21742910, train_accuracy: 0.9453, test_Accuracy: 0.9224
E